In [1]:
# includes
from PIL import Image
from skimage import io, transform

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import torch.optim as optim

In [2]:
# constansts

project_path = "/home/krys/POLIBUDA/Patterns_database/"


dots_path = project_path + "dots/dot"
paisley_path = project_path + "paisley/paisley"
qua_path = project_path + "quaterfoil/qua"
square_path = project_path + "squares/sq"
strips_path = project_path + "strips/pas"
traingles_path = project_path + "triangles/tr"

dots_target = torch.tensor([[float(1), float(0), float(0), float(0), float(0), float(0)]])
paisley_target = torch.tensor([[float(0), float(1), float(0), float(0), float(0), float(0)]])
qua_target = torch.tensor([[float(0), float(0), float(1), float(0), float(0), float(0)]])
square_target = torch.tensor([[float(0), float(0), float(0), float(1), float(0), float(0)]])
strips_target = torch.tensor([[float(0), float(0), float(0), float(0), float(1), float(0)]])
traingles_target = torch.tensor([[float(0), float(0), float(0), float(0), float(0), float(1)]])

paths = [
    dots_path,
    paisley_path,
    qua_path,
    square_path,
    strips_path,
    traingles_path
]

targets = [
    dots_target,
    paisley_target,
    qua_target,
    square_target,
    strips_target,
    traingles_target
]


def getImageTensor(path, idx):
    
    # loading image from local memory
    
    i_path = path + str(idx) + ".jpg"
    image = io.imread(i_path)
    
    return image
    

# transforms image to grayscale tensor
def myTransform(image):

    transformedImage =  tv.transforms.Compose([
                        tv.transforms.ToPILImage(),
                        tv.transforms.Grayscale(num_output_channels=1),
                        tv.transforms.ToTensor()
                      ])(image)
    
    transformedImage = transformedImage[0]

    return transformedImage

In [3]:
# includes
from PIL import Image
from skimage import io, transform

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import torch.optim as optim

# nn model using pytorch schematic
class PatternRecognitionNet(nn.Module):

    def __init__(self):
        super(PatternRecognitionNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 73 * 73, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 6)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
# creting net, lossfunction, optimizer    
pattNet = PatternRecognitionNet()
criterion = nn.MSELoss()
optimizer = optim.SGD(pattNet.parameters(), lr=0.3) # check learning rate

In [5]:
# training function
def train(image, target):
    
    optimizer.zero_grad()   # zero the gradient buffers
    input = image.view(-1,1,300,300)
    output = pattNet(input)
    print("Target: ",target)
    print("Output: ",output)
    loss = criterion(output, target)
    print("Loss calculation: " , loss)
    loss.backward()
    optimizer.step()    # Does the update

In [6]:
idx = 1
error_count = 0
while(1):
    try:
        
        for k in range(len(paths)):    
            print("============================================================\n")
            print("Training : " + str(idx) + " : " + str(k+1) + "/6")
            print("Path: " + paths[k])
            print("\n")
            image = getImageTensor(paths[k], idx)
            t_image = myTransform(image)
            train(t_image, targets[k])
            print("\n")
        
    except Exception as e:
        print("*******************************************************\n")
        print(e)
        print("\n")
        print("*******************************************************")
        
        error_count = error_count + 1
        
        if error_count > 2:    
            break 
        
    finally:
        idx = idx + 1


Training : 1 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


Target:  tensor([[1., 0., 0., 0., 0., 0.]])
Output:  tensor([[-0.0741, -0.0677,  0.0420,  0.0001, -0.0936, -0.1135]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1969, grad_fn=<MseLossBackward>)



Training : 1 : 2/6
Path: /home/krys/POLIBUDA/Patterns_database/paisley/paisley


Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[ 0.1375, -0.0154, -0.0857,  0.1299, -0.1978, -0.0486]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1859, grad_fn=<MseLossBackward>)



Training : 1 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[-1.2491e-04,  9.3715e-01,  1.5624e-01,  6.5299e-03, -1.9951e-01,
          1.6805e-01]], grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.2764, grad_fn=<MseLossBackward>)



Training : 1 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0.




Training : 5 : 2/6
Path: /home/krys/POLIBUDA/Patterns_database/paisley/paisley


Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[0.2274, 0.1025, 0.1143, 0.1448, 0.1504, 0.1861]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1581, grad_fn=<MseLossBackward>)



Training : 5 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[0.1791, 0.2025, 0.1037, 0.1204, 0.1405, 0.1561]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1558, grad_fn=<MseLossBackward>)



Training : 5 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[0.1577, 0.1768, 0.2164, 0.1090, 0.1203, 0.1304]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1547, grad_fn=<MseLossBackward>)



Training : 5 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor(

Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[0.2184, 0.1089, 0.1186, 0.1402, 0.1725, 0.1992]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1575, grad_fn=<MseLossBackward>)



Training : 9 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[0.1946, 0.2166, 0.1079, 0.1276, 0.1564, 0.1757]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1587, grad_fn=<MseLossBackward>)



Training : 9 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[0.1734, 0.1926, 0.2175, 0.1163, 0.1375, 0.1516]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1562, grad_fn=<MseLossBackward>)



Training : 9 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[0.1568, 0.1723, 0.1986, 0.2236, 0.1160, 0.1283]],
       grad_fn=<AddmmBackward>)





Training : 13 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[0.1983, 0.2187, 0.1217, 0.1372, 0.1490, 0.1703]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1548, grad_fn=<MseLossBackward>)



Training : 13 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[0.1754, 0.1945, 0.2209, 0.1231, 0.1348, 0.1508]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1546, grad_fn=<MseLossBackward>)



Training : 13 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[0.1566, 0.1715, 0.1972, 0.2248, 0.1140, 0.1296]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1575, grad_fn=<MseLossBackward>)



Training : 13 : 6/6
Path: /home/krys/POLIBUDA/Patterns_database/triangles/tr


Target:  tensor([[0., 0., 0., 0., 0., 1.]])
Output:  tensor




Training : 17 : 6/6
Path: /home/krys/POLIBUDA/Patterns_database/triangles/tr


Target:  tensor([[0., 0., 0., 0., 0., 1.]])
Output:  tensor([[0.1367, 0.1546, 0.1701, 0.1896, 0.2212, 0.1241]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1539, grad_fn=<MseLossBackward>)



Training : 18 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


Target:  tensor([[1., 0., 0., 0., 0., 0.]])
Output:  tensor([[0.1198, 0.1348, 0.1464, 0.1649, 0.1985, 0.2258]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1553, grad_fn=<MseLossBackward>)



Training : 18 : 2/6
Path: /home/krys/POLIBUDA/Patterns_database/paisley/paisley


Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[0.2203, 0.1191, 0.1353, 0.1499, 0.1672, 0.1922]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1550, grad_fn=<MseLossBackward>)



Training : 18 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  ten

...




Training : 1150 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[0.1921, 0.2134, 0.1260, 0.1400, 0.1556, 0.1729]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1150 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[0.1729, 0.1921, 0.2134, 0.1260, 0.1400, 0.1556]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1150 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[0.1556, 0.1729, 0.1921, 0.2134, 0.1260, 0.1400]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1150 : 6/6
Path: /home/krys/POLIBUDA/Patterns_database/triangles/tr


Target:  tensor([[0., 0., 0., 0., 0., 1.]])
Output:

Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[0.1729, 0.1921, 0.2134, 0.1260, 0.1400, 0.1556]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1154 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[0.1556, 0.1729, 0.1921, 0.2134, 0.1260, 0.1400]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1154 : 6/6
Path: /home/krys/POLIBUDA/Patterns_database/triangles/tr


Target:  tensor([[0., 0., 0., 0., 0., 1.]])
Output:  tensor([[0.1400, 0.1556, 0.1729, 0.1921, 0.2134, 0.1260]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1155 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


Target:  tensor([[1., 0., 0., 0., 0., 0.]])
Output:  tensor([[0.1260, 0.1400, 0.1556, 0.1729, 0.1921, 0.2134]],
       grad_fn=<AddmmBackwa

Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[0.1729, 0.1921, 0.2134, 0.1260, 0.1400, 0.1556]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1158 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[0.1556, 0.1729, 0.1921, 0.2134, 0.1260, 0.1400]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1158 : 6/6
Path: /home/krys/POLIBUDA/Patterns_database/triangles/tr


Target:  tensor([[0., 0., 0., 0., 0., 1.]])
Output:  tensor([[0.1400, 0.1556, 0.1729, 0.1921, 0.2134, 0.1260]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1159 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


Target:  tensor([[1., 0., 0., 0., 0., 0.]])
Output:  tensor([[0.1260, 0.1400, 0.1556, 0.1729, 0.1921, 0.2134]],
       grad_fn=<AddmmBackwa


Training : 1162 : 6/6
Path: /home/krys/POLIBUDA/Patterns_database/triangles/tr


Target:  tensor([[0., 0., 0., 0., 0., 1.]])
Output:  tensor([[0.1400, 0.1556, 0.1729, 0.1921, 0.2134, 0.1260]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1163 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


Target:  tensor([[1., 0., 0., 0., 0., 0.]])
Output:  tensor([[0.1260, 0.1400, 0.1556, 0.1729, 0.1921, 0.2134]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1163 : 2/6
Path: /home/krys/POLIBUDA/Patterns_database/paisley/paisley


Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[0.2134, 0.1260, 0.1400, 0.1556, 0.1729, 0.1921]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1163 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output

Target:  tensor([[0., 0., 0., 0., 0., 1.]])
Output:  tensor([[0.1400, 0.1556, 0.1729, 0.1921, 0.2134, 0.1260]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 1167 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


*******************************************************

No such file: '/home/krys/POLIBUDA/Patterns_database/dots/dot1167.jpg'


*******************************************************

Training : 1168 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


*******************************************************

No such file: '/home/krys/POLIBUDA/Patterns_database/dots/dot1168.jpg'


*******************************************************

Training : 1169 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


*******************************************************

No such file: '/home/krys/POLIBUDA/Patterns_database/dots/dot1169.jpg'


*******************************************************


In [7]:
PATH = project_path + "/modelV3.pt"
torch.save(pattNet.state_dict(), PATH)